In [1]:
import json
import urllib
import pandas as pd 
import numpy as np
from io import StringIO 
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import collections
import heapq
import geopy
from geopy.geocoders import Nominatim, GoogleV3
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json, string
import requests
import geocoder
from folium import Map, Marker, GeoJson, LayerControl
from ediblepickle import checkpoint
from tqdm import tqdm_notebook
import os 
import folium
from folium import plugins
%matplotlib inline

In [10]:
with open('coordinates.co','r') as f:
    out = f.readlines()
out = out[7:]
# create DataFrame
Node_data = pd.DataFrame([map(int,raw.strip("\n").strip('v ').split()) for raw in out],columns = ['Id_Node', 'Latitude', 'Longitude'])

#info_norm =(df[['Longitude','Latitude']] - df[['Longitude','Latitude']].mean())/df[['Longitude','Latitude']].std()
#sns.scatterplot(x = 'Longitude', y = 'Latitude'  , data = info_norm)

with open('distancegraph.gr','r') as f:
    out = f.readlines()
out = out[7:]
# create DataFrame
Distance_data = pd.DataFrame([map(int,i.strip('\n').strip("a ").split()) for i in out],columns =['Id_Node1', 'Id_Node2', 'd(Id_Node1,Id_Node2)'])

with open('traveltimegraph.gr','r') as f:
    out = f.readlines()
out = out[7:]
# create DataFrame
Travel_time_data = pd.DataFrame([map(int,i.strip('\n').strip("a ").split()) for i in out],columns =['Node1', 'Node2', 't(Id_Node1, Id_Node2)'])

I imported the data in the same way you (Lev)did but instead change everything i did that:

In [11]:
coordinates=Node_data
distance=Distance_data
traveltime=Travel_time_data

Delete variables 

In [20]:
del Node_data
del Distance_data
del Travel_time_data

In [16]:
distance.rename({'Id_Node1': 'Node1', 'Id_Node2': 'Node2', 'd(Id_Node1,Id_Node2)':'Distance'}, axis=1,inplace=True)
traveltime.rename({'t(Id_Node1, Id_Node2)': 'TimeTravel'}, axis=1,inplace=True)

Built a dictionary : node as key {connection node, distance, time distance, latitude, longitude)

In [ ]:
from collections import defaultdict
d = defaultdict(list)
for index in tqdm_notebook(distance.index, desc='Fill the Dictionary'):
    d[distance["Node1"].iloc[index]].append((distance["Node2"].iloc[index],distance["Distance"].iloc[index],traveltime["TimeTravel"].iloc[index],1))
#np.save('my_file.npy', d)

### Functionality 2 - Find the smartest Network!
### It takes in input: a set of nodes v = {v_1, ..., v_n}
One of the following distances function: t(x,y), d(x,y) or network distance (i.e. consider all edges to have weight equal to 1).
Implement an algorithm that returns the set of roads (edges) that enable the user to visit all the places. We want this set to be the ones whose sum of distances is minimum.

As a dummy example, a set of input could be {Colosseo, Piazza Venezia, Piazza del Popolo} and therefore the associated set of streets will be {Via dei Fori Imperiali, Via del Corso}.


#### It means find -> minimum spanning tree ->kruskal
#### we can consider as our weights the time or the distance 

pseudocode:
1. Sort all the edges in non-decreasing order of their weight
2. Pick the smallest edge. Check if it forms a cycle with the spanning tree formed so far. If cycle is not formed, include this edge. Else, discard it.
3. Repeat step #2 untile there are (V-1) edges in the spanning tree

NETWORKX
0

You are solving the Steiner tree problem, which is NP-hard. networkx provides an approximation algorithm to solve this problem: networkx.algorithms.approximation.steinertree.steiner_tree.

https://www.geeksforgeeks.org/steiner-tree/
https://gis.stackexchange.com/questions/307336/generating-steiner-tree-using-network-x-in-python

In [209]:
def spanning_tree_distance(a):
    print('Choose from which node start')
    node=int(input())
    mst = defaultdict(set)
    visited = set([node])
    edges = [(dist,node,to) for to,dist,_,_ in d[node]]
    heapq.heapify(edges)

    while edges:
        dist, frm, to = heapq.heappop(edges) #Pop and return the smallest item from the heap, maintaining the heap invariant.
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next, dist,_,_ in d[to]:
                if to_next not in visited:
                    heapq.heappush(edges, (dist, to, to_next)) #Push the value item into the heap, maintaining the heap invariant.
    return mst


In [157]:
def spanning_tree_time(d):
    print('Choose from which node start')
    node=int(input())   
    mst = defaultdict(set)
    visited = set([node])
    edges = [(time,node,to) for to,_,time,_ in d[node]]
    heapq.heapify(edges)

    while edges:
        time, frm, to = heapq.heappop(edges) #Pop and return the smallest item from the heap, maintaining the heap invariant.
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next,_,time,_ in d[to]:
                if to_next not in visited:
                    heapq.heappush(edges, (time, to, to_next)) #Push the value item into the heap, maintaining the heap invariant.
    return mst



In [158]:
def spanning_tree_weight(d):
    print('Choose from which node start')
    node=int(input())
    mst = defaultdict(set)
    visited = set([node])
    edges = [(weight,node,to) for to,_,_,weight in d[node]]
    heapq.heapify(edges)

    while edges:
        weight, frm, to = heapq.heappop(edges) #Pop and return the smallest item from the heap, maintaining the heap invariant.
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next,_,time,_ in d[to]:
                if to_next not in visited:
                    heapq.heappush(edges, (weight, to, to_next)) #Push the value item into the heap, maintaining the heap invariant.
    return mst


Frontend

In [370]:
from IPython.display import display, HTML
nodini=[]
print('Choose how many nodes consider for the spanning tree')
n=int(input())
print('Choose which nodes consider')
for i in range(0,n):
    nodini.append(int(input()))
print('Choose which method use for find the smartest Network:Time, Weight, Distance. p.s. If you are not interest just write "quit"') 

a=input().lower()
if a=='time':
    b=dict(spanning_tree_time(d))
    

elif a=='weight':
    b=dict(spanning_tree_weight(d))
    




elif a=='distance':
    b=dict(spanning_tree_distance(d))

    

elif a=='quit':
    print(':(')

else:
    print('What you said? please repeat!')






Choose how many nodes consider for the spanning tree
15
Choose which nodes consider
1
2
3
4
5
6
78
9
10
14
16
17
188
14343
324356
Choose which method use for find the smartest Network:Time, Weight, Distance. p.s. If you are not interest just write "quit"
Time
Choose from which node start
1


In [385]:
Values=[]
for i in range(len(nodini)):

    try:
        Values.append(b[nodini[i]])
    except:
          continue

In [387]:
len(Values)

13

In [372]:
result_spanning = pd.DataFrame(list(b.items()), columns=['Id_Node', 'Connection'])

In [373]:
lista = []
for i in range(len(Values)):
    lista.append(result_spanning[result_spanning['Connection']==Values[i]])

In [374]:
result_spanning_1 = pd.concat(lista)

In [375]:
df_final = pd.merge(result_spanning_1, coordinates, on='Id_Node', how='inner')
df_final_1 = pd.merge(result_spanning, coordinates, on='Id_Node', how='inner')

In [382]:
#fist longitude and then latitude, the resultino['longitude, latitude'] are divided by one milion because coordinates are something like 45.0003
map = folium.Map(location=[np.median((df_final['Longitude']/1000000).tolist()),
                           np.median((df_final['Latitude']/1000000).tolist())], default_zoom_start=15)
#keep the median to localize our data

for i in tqdm_notebook(range(0,len(df_final))):
    folium.CircleMarker(location = [((df_final['Longitude']/1000000).values)[i], 
                                                     ((df_final['Latitude']/1000000).values)[i]],
                                             number_of_sides = 4,
                                             radius = 10,
                                             weight = 8,
                                             color='red',
                                             icon='home',
                                             fill_opacity = 0.8).add_to(map)

#to add the edge between two nodes
for i in tqdm_notebook(range(0,len(df_final)-1)):
    folium.PolyLine(locations = [(((df_final['Longitude']/1000000).values)[i], 
                                  ((df_final['Latitude']/1000000).values)[i]), 
                                 (((df_final['Longitude']/1000000).values)[i+1], 
                                  ((df_final['Latitude']/1000000).values)[i+1])], 
                    line_opacity = 0.5,color='red').add_to(map)
folium.CircleMarker([((df_final['Longitude']/1000000).values)[i],((df_final['Latitude']/1000000).values)[i]],
                    radius=15, color='red').add_to(map)
#weight represent the dimension of the node
map.save("Spanning_tree.html")

#All the spanning_tree with the previous one
#this take two hours and half to execute the first loop ( of all the spanning tree!!)
#takes 5 hours to execute the second 
#fist longitude and then latitude, the resultino['longitude, latitude'] are divided by one milion because coordinates are something like 45.0003

#keep the median to localize our data
df_final_1=df_final_1[:2000]
for i in tqdm_notebook(range(0,len(df_final_1))):
    folium.features.RegularPolygonMarker(location = [((df_final_1['Longitude']/1000000).values)[i], 
                                                     ((df_final_1['Latitude']/1000000).values)[i]],
                                             number_of_sides = 4,
                                             radius = 4,
                                             weight = 8,
                                             color='green',
                                             fill_opacity = 0.8).add_to(map)

#to add the edge between two nodes
for i in tqdm_notebook(range(0,len(df_final_1)-1)):
    folium.PolyLine(locations = [(((df_final_1['Longitude']/1000000).values)[i], 
                                  ((df_final_1['Latitude']/1000000).values)[i]), 
                                 (((df_final_1['Longitude']/1000000).values)[i+1], 
                                  ((df_final_1['Latitude']/1000000).values)[i+1])], 
                    line_opacity = 0.2,color='green').add_to(map)
#weight represent the dimension of the node
map.save("Spanning_tree_2.html")

map

'reference'
https://deparkes.co.uk/2019/02/27/folium-lines-and-markers/